In [ ]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
wd = webdriver.Chrome(executable_path='C:\\Users\\erics\\Downloads\\Chrome Driver\\chromedriver.exe')
url = 'https://fbref.com/en/comps/9/history/Premier-League-Seasons'
wd.get(url)

In [ ]:
# Print HTML source code of the page
print(wd.page_source)

In [ ]:
# Print page title
print(wd.title)

In [ ]:
# Print current page url
print(wd.current_url)

In [ ]:
# Accessing an HTML element by it's text value
search_str = '2022-2023'
elmt = wd.find_element(By.LINK_TEXT, search_str)
print(type(elmt))

In [ ]:
# Accessing an HTML element text value
elmt.text

In [ ]:
# Accessing an HTML element attribute
elmt.get_attribute('href')

In [ ]:
# Running javascript code to scroll down
javaScript = "window.scrollBy(0, 850);"
wd.execute_script(javaScript)

In [ ]:
intLoc = wd.page_source.find(search_str)
print(wd.page_source[intLoc : intLoc + 9])

In [ ]:
wd.maximize_window()

### Retrieve all the rows in the table

In [ ]:
seasons = wd.find_elements_by_xpath('//*[@id="seasons"]/tbody/tr')

In [ ]:
# Iterating over all the rows retrieved from table
for i, x in enumerate(seasons):
    print(i)
    print(x.text)

In [ ]:
# Checking length of the list
len(seasons)

In [ ]:
# Finding irrelevant rows
for i, x in enumerate([season for season in seasons if not (season.text.find('Season'))]):
    print(i)
    print(x.text)

In [ ]:
# Removing irrelevant rows from the list by list comprehension
seasons = [season for season in seasons if season.text.find('Season')]

In [ ]:
# Checking length of the list
len(seasons)

In [ ]:
# Creating a list of links to the season table for each season for the history of premier league
plSList = [seasons[i].find_element_by_xpath('./child::*/a').get_attribute('href') for i in range(0, len(seasons))]

In [ ]:
plSList = plSList[1 : 21]

In [ ]:
for i, x in enumerate(plSList):
    print(str(i) + "\t" + x)

In [ ]:
wd.close()

In [ ]:
allSeasonsDF = pd.DataFrame([], columns=[
    'season', 'seasonRK', 'teamName', 
    'homeMP', 'homeW', 'homeD', 'homeL', 'homeGF', 'homeGA', 'homeGD', 'homePts', 'homePtsMP', 'homeXG', 
    'homeXGA', 'homeXGD', 'homeXGD90',
    'awayMP', 'awayW', 'awayD', 'awayL', 'awayGF', 'awayGA', 'awayGD', 'awayPts', 'awayPtsMP', 'awayXG', 
    'awayXGA', 'awayXGD', 'awayXGD90'
])

In [ ]:
def retrieveSeasonTable(df, seasonTableLink):
    wd = webdriver.Chrome(executable_path='C:\\Users\\erics\\Downloads\\Chrome Driver\\chromedriver.exe')
    wd.get(seasonTableLink)
    wd.maximize_window()
    wd.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
    homeAwayTable = wd.find_element_by_link_text("Home/Away")
    homeAwayTable.click()
    teamsRanks = wd.find_elements_by_xpath('/html/body/div[2]/div[6]/div[3]/div[4]/div[3]/table/tbody/tr')
    for team in teamsRanks:
        t = team.find_elements_by_xpath('./child::*')
        lst = ['2021-2022'] + [i.text for i in t]
        df = pd.concat([df, pd.Series(lst, index=df.columns).to_frame().T], ignore_index=True)
    wd.close()
    return df

In [ ]:
for lnk in plSList:
    allSeasonsDF = retrieveSeasonTable(allSeasonsDF, lnk)

In [ ]:
allSeasonsDF.to_excel('All Seasons Dataset.xlsx')